# 📓 The GenAI Revolution Cookbook

**Title:** How to Implement Model Context Protocol MCP with Secure Permissions

**Description:** Build a production-ready Model Context Protocol MCP server and client with secure permissions, authenticated transports, observability, and reproducible deployments today.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Why Secure MCP Servers Matter

When you expose file operations or other privileged actions to an LLM, you're handing untrusted prompts the keys to your system. Without strict input validation, scope enforcement, and audit trails, a single malicious or malformed prompt can read secrets, overwrite data, or traverse directories. MCP standardizes tool interfaces, but security is your responsibility.

This guide shows you how to build a **secure, read-only MCP server over stdio** with version negotiation, scope-based authorization, Pydantic-validated inputs, and JSON audit logs. By the end, you will:

- Validate protocol versions to ensure client-server compatibility.
- Enforce least-privilege scopes using environment-driven authorization.
- Emit structured JSON audit logs for compliance and debugging.
- Prevent path traversal, hidden file access, and oversized payloads.

We'll focus on stdio transport for local, isolated operation. HTTP, mTLS, and containerization are covered in separate tutorials.

---

## Why Use MCP for This Problem

Ad-hoc tool wrappers scatter validation, logging, and auth logic across your codebase. MCP provides:

- **Standardized schemas**: Tools declare inputs/outputs with JSON Schema, enabling automatic validation.
- **Explicit capabilities**: Clients and servers negotiate features up front, eliminating hidden assumptions.
- **Transport flexibility**: Start with stdio for local dev, scale to HTTP for multi-tenant deployments.

For a constrained file reader exposed to an LLM app, MCP lets you enforce read-only access, reject traversal attempts, and audit every invocation—without reinventing protocol logic.

---

## Core Concepts for This Use Case

**Tools**: Functions the LLM can invoke. Each tool has a name, description, and input schema. Our server exposes `read_file` (read-only) and `write_file` (privileged).

**Scopes**: Permissions required to invoke a tool. We use `files.read` and `files.write` to enforce least-privilege access.

**Protocol version**: MCP uses date-based versions (e.g., `2025-06-18`). Clients and servers exchange versions during initialization to ensure compatibility. For more on choosing the right model for your application and environment, check out our guide on [how to pick an LLM](/article/how-to-choose-an-ai-model-for-your-app-speed-cost-reliability).

**Audit logs**: Structured JSON records of every tool invocation, including inputs, outcomes, and errors. These logs support compliance, debugging, and anomaly detection.

---

## Setup

Install the MCP Python SDK and dependencies:

In [ ]:
!pip install mcp pydantic python-json-logger

Set up environment variables for scopes and data root:

In [ ]:
import os
os.environ["MCP_SCOPES"] = "files.read,files.write"
os.environ["DATA_ROOT"] = "./data"
os.environ["MCP_SERVER_ID"] = "file-server-1"

Create a sample data directory and file:

In [ ]:
from pathlib import Path
data_root = Path("./data")
data_root.mkdir(exist_ok=True)
(data_root / "greeting.txt").write_text("Hello from MCP!", encoding="utf-8")

---

## Building the Secure Server

### Step 1: Logging and Audit Helpers

Set up structured JSON logging to emit audit events with timestamps, server ID, and event details. This helper sanitizes strings to remove control characters and truncate for safe logging.

In [ ]:
import json
import logging
import re
from datetime import datetime, timezone
from pythonjsonlogger import jsonlogger

logger = logging.getLogger("mcp_server")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(jsonlogger.JsonFormatter("%(message)s"))
logger.addHandler(handler)

SERVER_ID = os.environ.get("MCP_SERVER_ID", "file-server-1")

def sanitize(value: str, max_len: int = 512) -> str:
    value = re.sub(r"[\x00-\x1f\x7f]", " ", value or "")
    return value[:max_len]

def audit_log(event: str, **fields):
    record = {
        "ts": datetime.now(timezone.utc).isoformat(),
        "server_id": SERVER_ID,
        "event": event,
        **fields,
    }
    logger.info(json.dumps(record))

### Step 2: Scope-Based Authorization

Define helpers to retrieve scopes from the environment and enforce them before tool execution. This prevents unauthorized access even in local stdio scenarios.

In [ ]:
from typing import Set

def get_scopes_from_env() -> Set[str]:
    scopes = os.environ.get("MCP_SCOPES", "")
    return {s.strip() for s in scopes.split(",") if s.strip()}

def require_scope(required: str, scopes: Set[str]):
    if required not in scopes:
        raise PermissionError(f"missing_scope:{required}")

### Step 3: Input Validation with Pydantic

Define strict schemas for tool inputs. These schemas reject hidden files, path traversal attempts, and oversized content.

In [ ]:
from pydantic import BaseModel, Field, field_validator

DATA_ROOT = Path(os.environ.get("DATA_ROOT", "./data")).resolve()

class ReadFileInput(BaseModel):
    filename: str = Field(..., min_length=1, max_length=128, pattern=r"^[A-Za-z0-9._-]+$")

    @field_validator("filename")
    @classmethod
    def no_hidden_files(cls, v: str) -> str:
        if v.startswith("."):
            raise ValueError("hidden files not allowed")
        return v

class WriteFileInput(BaseModel):
    filename: str = Field(..., min_length=1, max_length=128, pattern=r"^[A-Za-z0-9._-]+$")
    content: str = Field(..., max_length=10000)

    @field_validator("filename")
    @classmethod
    def no_hidden_files(cls, v: str) -> str:
        if v.startswith("."):
            raise ValueError("hidden files not allowed")
        return v

### Step 4: Implement Read-Only Tool

The `read_file` tool enforces the `files.read` scope, validates the path, and sanitizes output. It logs every invocation and redacts file content from logs.

In [ ]:
from mcp.server.fastmcp import FastMCP, Context

mcp = FastMCP("secure-file-server")

@mcp.tool()
def read_file(ctx: Context, input: ReadFileInput) -> str:
    scopes = get_scopes_from_env()
    try:
        require_scope("files.read", scopes)
        path = (DATA_ROOT / input.filename).resolve()
        if not str(path).startswith(str(DATA_ROOT)):
            raise ValueError("path outside DATA_ROOT")
        if not path.exists() or not path.is_file():
            raise FileNotFoundError("not found")
        content = path.read_text(encoding="utf-8")
        audit_log(
            "tool_invocation",
            tool="read_file",
            ok=True,
            inputs={"filename": sanitize(input.filename)},
            redactions=["content"],
        )
        safe_content = sanitize(content, max_len=10000)
        return safe_content
    except Exception as e:
        audit_log(
            "tool_invocation",
            tool="read_file",
            ok=False,
            error=str(e),
            inputs={"filename": sanitize(getattr(input, 'filename', 'invalid'))},
        )
        raise

### Step 5: Implement Write Tool

The `write_file` tool requires the `files.write` scope and performs the same path validation. It normalizes newlines and logs content length without exposing the full payload.

In [ ]:
@mcp.tool()
def write_file(ctx: Context, input: WriteFileInput) -> str:
    scopes = get_scopes_from_env()
    try:
        require_scope("files.write", scopes)
        path = (DATA_ROOT / input.filename).resolve()
        if not str(path).startswith(str(DATA_ROOT)):
            raise ValueError("path outside DATA_ROOT")
        path.parent.mkdir(parents=True, exist_ok=True)
        normalized = sanitize(input.content, max_len=10000).replace("\r\n", "\n")
        path.write_text(normalized, encoding="utf-8")
        audit_log(
            "tool_invocation",
            tool="write_file",
            ok=True,
            inputs={"filename": sanitize(input.filename), "content_len": len(normalized)},
        )
        return "ok"
    except Exception as e:
        audit_log(
            "tool_invocation",
            tool="write_file",
            ok=False,
            error=str(e),
            inputs={
                "filename": sanitize(getattr(input, 'filename', 'invalid')),
                "content_len": len(getattr(input, 'content', '')),
            },
        )
        raise

### Step 6: Start the Server

Log server startup with tool names and data root, then run the server using stdio transport.

In [ ]:
if __name__ == "__main__":
    audit_log("server_start", tools=["read_file", "write_file"], data_root=str(DATA_ROOT))
    mcp.run()

Save the complete server code to `server.py` for use by the client.

---

## Building the Client

This async client launches the server as a subprocess, negotiates protocol version, lists tools, and invokes them. It demonstrates scope enforcement by attempting a write with insufficient permissions.

In [ ]:
import asyncio
from mcp.client.stdio import StdioServer

PROTOCOL_VERSION = "2025-06-18"

async def main():
    env = os.environ.copy()
    env["MCP_SCOPES"] = "files.read,files.write"
    async with StdioServer.create("python", ["server.py"], env=env) as server:
        await server.initialize(
            protocol_version=PROTOCOL_VERSION,
            client_name="demo-client",
            client_version="1.0.0",
            capabilities={"tools": {"invoke": True, "list": True}},
        )
        tools = await server.list_tools()
        print("Tools:", json.dumps(tools, indent=2))

        result = await server.call_tool("read_file", {"input": {"filename": "greeting.txt"}})
        print("read_file result:", result)

asyncio.run(main())

To test scope enforcement, launch a new server subprocess with reduced scopes:

In [ ]:
async def test_scope_enforcement():
    env = os.environ.copy()
    env["MCP_SCOPES"] = "files.read"
    async with StdioServer.create("python", ["server.py"], env=env) as server:
        await server.initialize(
            protocol_version=PROTOCOL_VERSION,
            client_name="demo-client",
            client_version="1.0.0",
            capabilities={"tools": {"invoke": True, "list": True}},
        )
        try:
            await server.call_tool("write_file", {"input": {"filename": "new.txt", "content": "x"}})
        except Exception as e:
            print("write_file rejected:", e)

asyncio.run(test_scope_enforcement())

---

## Run and Evaluate

Start the server and client in separate cells or processes. The client will list tools, read `greeting.txt`, and demonstrate scope enforcement.

Tail JSON logs to verify audit events:

In [ ]:
import subprocess
subprocess.run(["tail", "-f", "/path/to/logs"], check=False)

Or capture logs programmatically and parse JSON records to confirm required fields (`ts`, `server_id`, `event`, `tool`, `ok`, `inputs`).

---

## Preventing Prompt Injection via Output Policy

Even read-only tools can become injection vectors if outputs are not sanitized. The `read_file` tool removes control characters and truncates content to prevent malicious payloads from corrupting downstream prompts. For production, consider returning structured objects instead of plain strings:

In [ ]:
return {"content": safe_content, "path": str(path), "source": "file"}

This allows prompting layers to render safely and apply downstream policies. For a deeper dive into how prompt structure and information placement affect model behavior, see our article on [placing critical info in long prompts](/article/lost-in-the-middle-placing-critical-info-in-long-prompts).

---

## Protocol Version Negotiation

Clients and servers should exchange protocol versions and capabilities up front. The MCP spec uses date-based versions. Pick a target like `2025-06-18` and reject mismatches. Capabilities advertise what each side supports, enabling graceful fallback and explicit version checks. This yields deterministic behavior and eliminates hidden assumptions across environments. If you're interested in how context limitations can affect model reliability and what strategies can help, see our breakdown of [context rot and why LLMs "forget" as their memory grows](/article/context-rot-why-llms-forget-as-their-memory-grows-3).

To enforce version checks on the server, hook the `initialize` method in your MCP implementation to validate `protocolVersion` and reject unsupported clients. Consult the MCP spec at https://modelcontextprotocol.io/spec for details.

---

## Conclusion

You've built a secure MCP server that enforces least-privilege scopes, validates inputs with Pydantic, and emits structured audit logs. This foundation supports safe LLM tool integration without risking traversal, hidden file access, or unaudited writes.

**Next steps**:
- Expose the server over HTTP behind mTLS or bearer token authentication (covered in a separate tutorial).
- Add change detection alerts by polling tool definitions and diffing snapshots (see `watch_tools.py` example in the code improvements).